# 4_7_Deriving_New_Variables

### This script contains the following points:

### 1. Importing Libaries

### 2. User Defined Function

### 3. Loc Method- Sampled Dataframe

### 4. Loc Method - Whole Dataframe - ords_prods_merge

### 5. If-Statements with For-Loops

### 6. “Busiest days” (plural) flag

### 7. Check the values of this new column for accuracy

### 8. Busiest hours of the day

### 9. Print the frequency for this new column

### 10. Export new Dataframe as Pickle

# 1. Importing Libaries

In [57]:
# Import libaries
import pandas as pd
import numpy as np
import os

In [58]:
# Import data path
path = r'/content/drive/MyDrive/Careerfoundary'

In [59]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [60]:
df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,First Order,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both


In [61]:
# Create a subset of the first 1000 line
df = df_ords_prods_merge[:1000000]

In [62]:
df.shape

(1000000, 15)

In [63]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,First Order,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both


# 2. User Defined Function

In [64]:
# Defining labels for your price range
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'


In [65]:
# Define your price_label function and apply it to the subset
df['price_range'] = df.apply(price_label, axis=1)

<ipython-input-65-08dc1098de0a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [66]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    756450
Low-range product    243550
Name: count, dtype: int64

In [67]:
#most expensive product in the subset
df['prices'].max()

14.8

# 3. Loc Method- Sampled Dataframe

In [68]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

<ipython-input-68-fc7ad2a718ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [69]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [70]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [71]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    756450
Low-range product    243550
Name: count, dtype: int64

# 4. Loc Method - Whole Dataframe - ords_prods_merge

In [72]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [73]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [74]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [75]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

# 5. If-Statements with For-Loops

In [76]:
# We want to find which days are busiest in terms of order to see what products are being bought on the busiest and slowest days.

In [77]:
# Lets see the order counts per day
df_ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [78]:
# create loop with busiest, least busy and regular busy order days

result = []

for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")


In [79]:
result #entry for every row within your dataframe

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Reg

In [80]:
# combine results columnn with ords_prods_merge dataframe

df_ords_prods_merge['busiest_day'] = result

In [81]:
df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day
0,2539329,1,1,2,8,NaN,First Order,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy


In [82]:
df_ords_prods_merge['busiest_day']. value_counts(dropna = False)

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

# 6. “Busiest days” (plural) flag - two busiest days of the week - they’d also like to know the two slowest days. Create a new column.

In [83]:
# Firstly we will find our the two busiest days and the two slowest day.
df_ords_prods_merge['orders_day_of_week']. value_counts(dropna = False)


orders_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [84]:
# The 2 busiests days = 0&1 and the two lowest days = 3&4, I will use the loc method

In [86]:
# I chose to use if statements with loops
result_1 = []

for value in df_ords_prods_merge["orders_day_of_week"]:
    if value == 0 or value == 1:
        result_1.append("Busiest days")
    elif value == 3 or value == 4:
        result_1.append("Slowest days")
    else:
        result_1.append("Regular busy")

In [87]:
#combined the results column with df_ord_prods_merge table and called it Busiest_Days(plural)
df_ords_prods_merge['Busiest_Days(plural)'] = result_1

In [88]:
df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_Days(plural)
0,2539329,1,1,2,8,NaN,First Order,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regular busy
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days


In [93]:
df_ords_prods_merge ['Busiest_Days(plural)']. value_counts (dropna = False)

Busiest_Days(plural)
Regular busy    12916111
Busiest days    11864412
Slowest days     7624336
Name: count, dtype: int64

# 7. Check the values of this new column for accuracy

In [99]:
# Find the total frequency for Busiest Days (0 & 1)
freq1 = df_ords_prods_merge['orders_day_of_week'].value_counts()[0]
freq2 = df_ords_prods_merge['orders_day_of_week'].value_counts()[1]


In [96]:
total_frquency_busiest_days = freq1 + freq2

In [97]:
total_frquency_busiest_days

11864412

In [100]:
# Find the total frequency for Slowest Days (3 & 4)
freq3 = df_ords_prods_merge['orders_day_of_week'].value_counts()[3]
freq4 = df_ords_prods_merge['orders_day_of_week'].value_counts()[4]


In [101]:
total_frquency_slowest_days = freq3 + freq4

In [102]:
total_frquency_slowest_days

7624336

In [103]:
# Example way to test if the columns as correct - I have totaled the 0 and 1 frequencies, They add up to busiest days total in the new column, I have done the same for slowest days also.

# 8. Busiest hours of the day

When too many users make Instacart orders at the same time, the app freezes. The senior technical officer at Instacart wants you to identify the busiest hours of the day. Rather than by hour, they want periods of time labeled “Most orders,” “Average orders,” and “Fewest orders.” Create a new column containing these labels called “busiest_period_of_day.”

In [104]:
# identify the busiest hours of the day
df_ords_prods_merge['order_hour_of_day']. value_counts (dropna = False)


order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

In [105]:
# I will split the hours into thirds (8 per category):
# The 'Most orders' will be - 10,11,14,15,13,12,16,9
# The 'Average orders' will be - 17,8,18,19,20,7,21,22
# The 'Fewest orders' will be- 23,6,0,1,5,2,4,3

In [106]:
# If statement with loops
result_3 = []

for value in df_ords_prods_merge['order_hour_of_day']:
  if value in [10,11,14,15,13,12,16,9]:
    result_3.append ('Most orders')
  elif value in [23,6,0,1,5,2,4,3]:
    result_3.append ('Fewest orders')
  else:
    result_3.append('Average orders')



In [107]:
#combined the results column with df_ord_prods_merge table and called it busiest_period_of_day
df_ords_prods_merge['busiest_period_of_day'] = result_3

# 9. Print the frequency for this new column

In [108]:
# Frequency of the new column
df_ords_prods_merge ['busiest_period_of_day']. value_counts (dropna = False)

busiest_period_of_day
Most orders       21118071
Average orders     9997651
Fewest orders      1289137
Name: count, dtype: int64

In [109]:
df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_Days(plural),busiest_period_of_day
0,2539329,1,1,2,8,NaN,First Order,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regular busy,Average orders
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders


In [111]:
df_ords_prods_merge.shape  # Shape of updated dataframe


(32404859, 19)

# 10. Export new Dataframe as Pickle

In [112]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))